In [209]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import importlib
import download_clean_data as dc
import ipca
import metrics
import kernel_regression as kr
importlib.reload(dc) 
importlib.reload(ipca)
importlib.reload(metrics)
importlib.reload(kr)

<module 'kernel_regression' from '/Users/matteoferrazzi/Documents/GitHub/ML-project-2/kernel_regression.py'>

In [20]:
folder_path = "/Users/matteoferrazzi/Documents/GitHub/ML-project-2/Data/monthly_data"

In [21]:
N = 100
date = 20150000
data, ret = dc.download_clean_data(folder_path, date, N)

In [22]:
x = []
for t in range(len(ret)-1):
    x.append(data[t].values.T@ret[t+1].values/len(ret[0]))

x_cov = np.sum([x[i].reshape((-1,1))@x[i].reshape((1,-1)) for i in range(len(x))], axis = 0)
eigval_x, eigvec_x = np.linalg.eig(x_cov)

idx = np.argsort(np.abs(eigval_x))[::-1]
sort_eigvec_x = eigvec_x[:,idx].real
k = 5
gamma_first = sort_eigvec_x[:,:k]

In [152]:
max_iter = 100
gamma, f_list = ipca.ipca(data, ret, gamma_first.copy(), max_iter)
W = np.eye(N)
W_list = [W]*(len(data)-1)
lambda1 = 0.1
lambda2 = 0.1
gamma_reg_w, f_list_reg_w = ipca.ipca_reg_w(data, ret, gamma_first.copy(), max_iter, lambda1, lambda2, W_list)

In [154]:
print(metrics.total_R_squared(ret, data, gamma, f_list),metrics.pred_R_squared(ret, data, gamma, f_list))

0.153357855168212 -0.004563129725073667


In [153]:
print(metrics.total_R_squared(ret, data, gamma_reg_w, f_list_reg_w),metrics.pred_R_squared(ret, data, gamma_reg_w, f_list_reg_w))

0.15167256959390363 -0.0028241664513473097


In [192]:
data2 = data.copy()
data2 = np.array(np.array(data2).reshape(72*100,94)) #flatten data, build X
tk = 1
l = 1
K = kr.K_LR(data2, tk, l)

In [150]:
Omega1=np.eye((len(data)-1)*N)
Omega2=np.eye(N)
f_list_kr, v_kr, Q_kr = kr.kernel_regression(data, ret, f_list.copy(), lambda1, lambda2, Omega1, Omega2, 100, N, K)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [151]:
print(metrics.total_R_squared_kr(ret, v_kr, Q_kr))

0.15172992757550952


In [207]:
m_hat = 10
L, B = kr.pivoted_chol(K, m_hat)

1
2
3
4
5
6
7
8
9


In [194]:
f_list_kr_LR, v_kr_LR = kr.kernel_regression_LR(data, K, B, ret, f_list.copy(), lambda1, lambda2, Omega2, 100, m_hat, N)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [210]:
print(metrics.total_R_squared_kr_LR(ret, B, K, v_kr_LR, f_list_kr_LR))

0.001358626838706134
